# Gentle Introduction to Pydantic v2

## Import module

In [1]:
# !pip install pydantic

In [2]:
import pydantic

pydantic.__version__

'2.11.2'

## Data Type Validation without Pydantic

In [3]:
class User:
    def __init__(self, id: int, name: str = "Jane Doe"):
        if not isinstance(id, int):
            raise TypeError(f"Expected id to be an int, got {type(id).__name__}")

        if not isinstance(name, str):
            raise TypeError(f"Expected name to be a str, got {type(name).__name__}")

        self.id = id
        self.name = name

In [ ]:
user = User(id="abc")

In [ ]:
try:
    user = User(id="abc")
except TypeError as e:
    print(e)

## Data Type Validation with Pydantic

### Case 1: Simple Data Type Validation

In [1]:
from pydantic import BaseModel
from datetime import date


class User(BaseModel):
    id: int
    name: str
    birth_date: date
    hobbies: list[str]

In [ ]:
user = User(id=123, name="andi", birth_date="2020-01-25", hobbies=["reading", "coding"])
# user = User(id="123", name="andi", birth_date="2020-01-25", hobbies=["reading", "coding"])
# user = User(id="123Z", name="andi", birth_date="2020-01-25", hobbies=["reading", "coding"])
# user = User(id=123, name="andi", birth_date="20-01-2020", hobbies=["reading", "coding"])

# user = User(id=123)

user

### Case 2: Default Value

In [14]:
from pydantic import BaseModel
from datetime import date


class User(BaseModel):
    id: int
    name: str = "Jane Doe"
    birth_date: date = None
    hobbies: list[str] = []

In [ ]:
user = User(id=123)
# user = User(id="Andi")

# user = User(id="123", name="andi")
# user = User(id="123", name=None)

# user = User(id="123", name="Andi", birth_date="2024-01-01")
# user = User(id="123", hobbies=["reading", "coding"])

user

### Case 3: Multiple Data Types

In [37]:
from pydantic import BaseModel
from datetime import date


class User(BaseModel):
    id: int
    name: str | int | None
    birth_date: date | None
    hobbies: list[str | int] = []

In [ ]:
# user = User(id=123)

# user = User(id="123", name="Andi", birth_date=None)
# user = User(id="123", name=888, birth_date=None)

# user = User(id="123", name=None, birth_date=None, hobbies=[1, "2", 3])

user

### Case 4: Exception Handling

In [34]:
from pydantic import BaseModel
from datetime import date


class User(BaseModel):
    id: int
    name: str
    birth_date: date
    hobbies: list[str]

In [ ]:
from pydantic import ValidationError

try:
    user = User(
        id="123Z", name="andi", birth_date="2020-01-25", hobbies=["reading", "coding"]
    )
except ValidationError as e:
    print(e)

## Dump to JSON

In [4]:
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name: str = "Jane Doe"

In [5]:
user = User(id=123, name="John Doe")

In [20]:
dumped = user.model_dump()
dumped

# type(dumped)

# import json
# with open("user.json", "w") as f:
#     json.dump(dumped, f)

In [18]:
dumped = user.model_dump_json()
dumped

# type(dumped)

# with open("user.json", "w") as f:
#     f.write(dumped)

## Load from JSON

In [11]:
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name: str = "John Doe"

In [ ]:
with open("user.json", "r") as f:
    json_str = f.read()

user = User.model_validate_json(json_str)
user

## Model Schema

In [18]:
from pydantic import BaseModel


class User(BaseModel):
    id: int
    name: str = "John Doe"

In [ ]:
user.model_json_schema()

In [ ]:
from pprint import pprint

pprint(user.model_json_schema())

## Nested Models

In [26]:
from pydantic import BaseModel


class Food(BaseModel):
    name: str
    price: float
    ingredients: list[str] = None


class Restaurant(BaseModel):
    name: str
    location: str
    foods: list[Food]

In [ ]:
restaurant_instance = Restaurant(
    name="Tasty Bites",
    location="123, Flavor Street",
    foods=[
        {
            "name": "Cheese Pizza",
            "price": 12.50,
            "ingredients": ["Cheese", "Tomato Sauce", "Dough"],
        },
        {"name": "Veggie Burger", "price": 8.99},
    ],
)

restaurant_instance

In [ ]:
from pprint import pprint

restaurant_instance.model_dump()
# pprint(restaurant_instance.model_dump())

## Advanced Data Type Validation

In [ ]:
# !pip install pydantic[email]

In [23]:
from pydantic import BaseModel
from pydantic import EmailStr, PositiveInt, HttpUrl
from pydantic import conlist


class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str


class Employee(BaseModel):
    name: str
    position: str
    email: EmailStr


class Owner(BaseModel):
    name: str
    email: EmailStr


class Restaurant(BaseModel):
    name: str
    owner: Owner
    address: Address
    employees: conlist(Employee, min_length=2)
    number_of_seats: PositiveInt
    delivery: bool
    website: HttpUrl

In [ ]:
restaurant_instance = Restaurant(
    name="Tasty Bites",
    owner={"name": "John Doe", "email": "john.doe@example.com"},
    address={
        "street": "123, Flavor Street",
        "city": "Tastytown",
        "state": "TS",
        "zip_code": "12345",
    },
    employees=[
        {"name": "Jane Doe", "position": "Chef", "email": "jane.doe@example.com"},
        {"name": "Mike Roe", "position": "Waiter", "email": "mike.roe@example.com"},
    ],
    number_of_seats=50,
    delivery=True,
    website="https://tastybites.com",
)

## Pydantic Field

In [ ]:
from pydantic import BaseModel
from pydantic import Field


class User(BaseModel):
    name: str
    # name: str = Field(...)


# user = User()
user = User(name="John Doe")
user

In [ ]:
from pydantic import BaseModel, Field


class User(BaseModel):
    name: str = "John Doe"
    # name: str = Field(default="John Doe")
    # name: str = Field("John Doe")


user = User()
user

In [ ]:
from uuid import uuid4

from pydantic import BaseModel, Field


class User(BaseModel):
    id: str = Field(default_factory=lambda: uuid4().hex)


user = User()
user

## Field aliases

In [ ]:
from pydantic import BaseModel, Field


class User(BaseModel):
    name: str = Field(alias="username")
    id: int = Field(alias="user_id")


user = User(user_id=123, username="johndoe")
user

In [ ]:
user.model_dump(by_alias=True)

## Field constraint

### Alternative 1

In [ ]:
from pydantic import BaseModel, Field, EmailStr, SecretStr


class User(BaseModel):
    email: EmailStr

    username: str = Field(min_length=3, max_length=10, pattern=r"^\w+$")
    # username: str = Field(min_length=3, max_length=10, pattern=r"^\w+$", alias="user_name")
    age: int = Field(gt=0, le=120)
    height: float = Field(gt=0.0)
    favorite_numbers: list[int] = Field(min_items=1)
    is_active: bool = True
    password: SecretStr

In [ ]:
user_instance = User(
    username="john_doe",
    # user_name="john_doe",
    age=30,
    height=5.9,
    weight=160.5,
    email="john.doe@example.com",
    password="securepassword",
    balance=9999.99,
    favorite_numbers=[1, 2, 3],
)

user_instance

### Alternative 2

In [40]:
from pydantic import BaseModel, Field, EmailStr, SecretStr
from pydantic import conint, confloat, conlist, constr


class User(BaseModel):
    email: EmailStr
    username: constr(min_length=3, max_length=10, pattern=r"^\w+$")
    # username: constr(min_length=3, max_length=10, pattern=r"^\w+$") = Field(alias="user_name")
    age: conint(gt=0, le=120)
    height: confloat(gt=0.0)
    favorite_numbers: conlist(int, min_length=1)

    is_active: bool = True
    password: SecretStr

In [ ]:
user_instance = User(
    username="john_doe",
    # user_name="john_doe",
    age=30,
    height=5.9,
    weight=160.5,
    email="john.doe@example.com",
    password="securepassword",
    balance=9999.99,
    favorite_numbers=[1, 2, 3],
)

user_instance

### More on `SecretStr`

In [ ]:
user_instance.password
# user_instance.password.get_secret_value()

## Strict mode

In [ ]:
from pydantic import BaseModel


class User(BaseModel):
    id: int
    username: str


data_dict = {"id": " 42 ", "username": "john_doe"}
User.model_validate(data_dict)

In [ ]:
User.model_validate(data_dict, strict=True)

## Pydantic Settings

In [ ]:
# !pip install pydantic-settings

In [ ]:
from pydantic import Field
from pydantic_settings import BaseSettings


class Config(BaseSettings):
    auth_key: str
    api_key: str = Field(alias="my_api_key")


cfg = Config()

In [ ]:
import os
from pydantic import Field, AliasChoices
from pydantic_settings import BaseSettings

os.environ["AUTH_KEY"] = "test_auth_key"
os.environ["MY_API_KEY"] = "test"
os.environ["ENV2"] = "https://mysuperurl.com"
os.environ["ENV1"] = "https://mysuperurl.net"


class Config(BaseSettings):
    service_name: str = Field(default="default")
    auth_key: str
    api_key: str = Field(alias="my_api_key")
    url: str = Field(validation_alias=AliasChoices("env1", "env2"))


cfg = Config()
cfg
# cfg.model_dump()

In [ ]:
import os
from pydantic import Field
from pydantic_settings import BaseSettings, SettingsConfigDict

# Set environment variables with the prefix
os.environ["PRODUCTION_AUTH_KEY"] = "test_auth_key"
os.environ["PRODUCTION_MY_API_KEY"] = "test"
os.environ["PRODUCTION_ENV2"] = "https://mysuperurl.com"


class Config(BaseSettings):
    model_config = SettingsConfigDict(env_prefix="production_")

    service_name: str = Field(default="default")
    auth_key: str
    api_key: str = Field(alias="my_api_key")
    url: str = Field(validation_alias=AliasChoices("env1", "env2"))


cfg = Config()
cfg.model_dump()

In [ ]:
from pydantic import Field
from pydantic_settings import BaseSettings, SettingsConfigDict


class Config(BaseSettings):
    model_config = SettingsConfigDict(
        env_file=".env", env_file_encoding="utf-8", extra="ignore"
    )

    service_name: str = Field(default="default")
    auth_key: str
    api_key: str = Field(alias="my_api_key")


cfg = Config()
cfg.model_dump()

## Computed Field

In [ ]:
from pydantic import BaseModel, computed_field
from datetime import datetime


class Person(BaseModel):
    name: str
    birth_year: int

    @computed_field
    def age(self) -> int:
        current_year = datetime.now().year
        return current_year - self.birth_year


Person(name="John Doe", birth_year=2000)

In [ ]:
from pydantic import BaseModel, computed_field


class User(BaseModel):
    first_name: str
    last_name: str

    @computed_field
    def full_name(self) -> str:
        return f"{self.first_name} {self.last_name}"

    @computed_field
    def email(self) -> str:
        return f"{self.first_name.lower()}.{self.last_name.lower()}@example.com"


user = User(first_name="John", last_name="Doe")
user

In [ ]:
from pydantic import BaseModel, ValidationError, field_validator
from datetime import datetime


class Person(BaseModel):
    name: str
    birth_year: int

    @computed_field
    def age(self) -> int:
        current_year = datetime.now().year
        return current_year - self.birth_year


Person(name="John Doe", birth_year=2006)

## Field Validator

In [1]:
from pydantic import BaseModel, EmailStr, field_validator


class Owner(BaseModel):
    name: str
    email: EmailStr

    @field_validator("name")
    @classmethod
    def name_must_contain_space(cls, v: str) -> str:
        if " " not in v:
            raise ValueError("Owner name must contain a space")
        return v.title()
        # return v.upper()

In [ ]:
try:
    owner_instance = Owner(name="john doe", email="john.doe@example.com")
    # owner_instance = Owner(name="johndoe", email="john.doe@example.com")
    # owner_instance = Owner(name="john doe")
    print(owner_instance)
except ValueError as e:
    print(e)

In [ ]:
from pydantic import BaseModel, ValidationError, field_validator
from datetime import datetime


class Person(BaseModel):
    name: str
    birth_year: int

    @field_validator("birth_year")
    @classmethod
    def validate_age(cls, v: int) -> int:
        current_year = datetime.now().year
        if current_year - v < 18:
            raise ValueError("Person must be 18 years or older")
        return v


try:
    print(Person(name="John Doe", birth_year=2016))
except ValidationError as e:
    print(e)

## Model Validator

In [4]:
from typing import Any
from pydantic import BaseModel, EmailStr, ValidationError, model_validator


class Owner(BaseModel):
    name: str
    email: EmailStr

    @model_validator(mode="before")
    @classmethod
    def check_sensitive_info_omitted(cls, data: Any) -> Any:
        if isinstance(data, dict):
            if "password" in data:
                raise ValueError("password should not be included")
            if "card_number" in data:
                raise ValueError("card_number should not be included")
        return data

    @model_validator(mode="after")
    def check_name_contains_space(self) -> "Owner":
        if " " not in self.name:
            raise ValueError("Owner name must contain a space")
        return self

In [ ]:
try:
    owner_instance = Owner(name="John Doe", email="john.doe@example.com")

    # owner_instance = Owner(
    #     name="John Doe", email="john.doe@example.com", password="password123"
    # )

    # owner_instance = Owner(name="JohnDoe", email="john.doe@example.com")

    print(owner_instance)
except ValidationError as e:
    print(e)